In [2]:
from PIL import Image
import numpy as np
from typing import Tuple, Union
import random

In [3]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray.astype(np.uint8)

In [4]:
maze_easy = rgb2gray(np.array(Image.open('maze_easy.png')))

In [5]:
Image.fromarray(maze_easy>0).save('maze_easy_bin.png')

In [6]:
maze_easy.shape

(316, 126)

In [7]:
class Node:
    def __init__(self, positionX: int, positionY: int):
        self.positionX = positionX
        self.positionY = positionY
        self.children = []
        self.parent = None

In [16]:
class RRTAlgo:
    def __init__(self, start: Node, goal: Node, num_itr: int, grid: np.ndarray, step_size: int) -> None:
        self.start = start
        self.goal = goal
        self.num_itr = num_itr
        self.nearest_node = None
        self.grid = grid
        self.step_size = step_size
        self.path_distance = 0
        self.nearest_distance = 1000000
        self.num_nodes = 0
        self.nodes = []

    def add_child(self, positionX, positionY):
        if self.distance(self.goal, np.array([positionX, positionY])) < self.step_size:
            print('HI')
            self.nearest_node.children.append(self.goal)
            self.goal.parent = self.nearest_node
        else:
            temp_node = Node(positionX, positionY)
            self.nearest_node.children.append(temp_node)
            temp_node.parent = self.nearest_node

    def sample_point(self):
        position = [random.randint(1, self.grid.shape[1]), random.randint(1, self.grid.shape[0])]
        return np.array(position)
    
    def steer_to_point(self, start: Node, end):
        offset = self.step_size * self.unit_vector(start, end)
        point = np.array([start.positionX + offset[0], start.positionY + offset[1]]).astype(np.int64)
        if point[0] >= self.grid.shape[1]:
            point[0] = self.grid.shape[1] - 1
        if point[1] >= self.grid.shape[0]:
            point[1] = self.grid.shape[0] - 1
        return point
    
    def check_obstacle(self, start: Node, end):
        u_hat = self.unit_vector(start, end)
        test_point = np.array([0.0,0.0])
        for i in range(self.step_size):
            test_point[0] = start.positionX + int(i*u_hat[0])
            test_point[1] = start.positionY + int(i*u_hat[1])
            if test_point[0] >= self.grid.shape[1]:
                test_point[0] = self.grid.shape[1] - 1
            if test_point[1] >= self.grid.shape[0]:
                test_point[1] = self.grid.shape[0] - 1
            if not self.grid[(test_point.astype(np.int64)[1])][(test_point.astype(np.int64)[0])]:
                return True
        return False

    def unit_vector(self, start: Node, end):
        v = np.array([end[0] - start.positionX, end[1] - start.positionY])
        u_hat = v / np.linalg.norm(v)
        return u_hat
    
    def find_nearest(self, root, point):
        if not root:
            return
        dist = self.distance(root, point)
        if dist < self.nearest_distance:
            self.nearest_node = root
            self.nearest_distance = dist
        
        for child in root.children:
            self.find_nearest(child, point)

    def distance(self, point_1: Node, point_2):
        dist = np.sqrt((point_1.positionX - point_2[0])**2 + (point_1.positionY - point_2[1])**2)
        return dist

    def goal_found(self, point):
        return self.distance(self.goal, point) < self.step_size

    def reset_nearest_values(self):
        self.nearest_distance = 1000000
        self. nearest_node = None

    def retrace_RRTPath(self, goal: Node):
        if goal.positionX == self.start.positionX and goal.positionY == self.start.positionY:
            return

        self.num_nodes += 1
        current_point = np.array([goal.positionX, goal.positionY])
        self.nodes.insert(0, current_point)
        self.path_distance += self.step_size
        self.retrace_RRTPath(goal.parent)


In [9]:
start = Node(maze_easy.shape[0] - 1, int(maze_easy.shape[1]/4))
end = Node(maze_easy.shape[0] - 1, int(maze_easy.shape[1]* 3 / 4))

In [17]:
rrt = RRTAlgo(start, end, 500, maze_easy, 50)

In [19]:
for i in range(rrt.num_itr):
    rrt.reset_nearest_values()
    print("Iteration: ", i)
    point= rrt.sample_point()
    rrt.find_nearest(rrt.start, point)
    new = rrt.steer_to_point(rrt.nearest_node, point)
    _bool = rrt.check_obstacle(rrt.nearest_node, new)
    if not _bool:
        rrt.add_child(new[0], new[1])
        print('Currently at: ', new)
        if (rrt.goal_found(new)):
            rrt.add_child(end.positionX, end.positionY)
            print("Goal Found!!")
            break

rrt.retrace_RRTPath(rrt.goal)        
rrt.nodes.insert(0, start)
print("Number of Nodes: ", rrt.num_nodes)
print("Path Distance: ", rrt.path_distance)
print("Nodes: ", rrt.nodes)

Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Itera

AttributeError: 'NoneType' object has no attribute 'positionX'

In [12]:
rrt.goal.positionX, rrt.goal.positionY

(315, 94)

In [13]:
rrt.goal.parent

In [14]:
rrt.nodes

[array([315,  94])]

In [15]:
rrt.nearest_node.positionX, rrt.nearest_node.positionY

(315, 31)

In [ ]:
rrt.nearest_node.children

[]

In [ ]:
rrt.grid.shape

(316, 126)

In [2]:
range(8.0)

TypeError: 'float' object cannot be interpreted as an integer